## 1. Importar Librerías

Importamos:
- `ChatAgent`: Agente de chat básico
- `AzureOpenAIChatClient`: Cliente de Azure OpenAI
- `json`: Serialización JSON
- `tempfile`: Directorio temporal para el ejemplo

In [ ]:
import os
import asyncio
import json
import tempfile
from azure.identity import AzureCliCredential
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from dotenv import load_dotenv

## 2. Cargar Configuración

Cargamos variables de entorno:
- `AZURE_OPENAI_ENDPOINT`: Endpoint de Azure OpenAI
- `MODEL`: Nombre del deployment del modelo

In [ ]:
load_dotenv()

print(f"✅ Configuración cargada")
print(f"Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"Modelo: {os.getenv('MODEL')}")

## 3. Crear el Chat Agent

### ChatAgent vs Agent:
- `ChatAgent`: Wrapper de más alto nivel para conversaciones
- Maneja threads automáticamente
- Simplifica el estado de conversación

### Parámetros:
- **chat_client**: Cliente de Azure OpenAI
- **name**: Nombre del asistente
- **instructions**: Comportamiento del agente

In [ ]:
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
        credential=AzureCliCredential(),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("MODEL")
    ),
    name="Assistant",
    instructions="You are a helpful assistant."
)

print("✅ ChatAgent creado")
print("   - Name: Assistant")
print("   - Instructions: You are a helpful assistant.")

## 4. Crear un Thread de Conversación

### ¿Qué es un Thread?
Un thread representa una **sesión de conversación** con:
- Historial de mensajes
- Contexto acumulado
- Estado de la conversación

### Crear un nuevo thread:
```python
thread = agent.get_new_thread()
```

In [ ]:
thread = agent.get_new_thread()

print("✅ Thread creado")
print(f"   Thread ID: {thread.id if hasattr(thread, 'id') else 'N/A'}")

## 5. Primera Conversación

Vamos a tener una primera interacción con el agente.

### Mensaje 1:
"Tell me a short pirate joke."

El agente responderá con un chiste pirata.

In [ ]:
async def first_conversation():
    """Primera parte de la conversación"""
    
    print("\n" + "="*70)
    print("PRIMERA CONVERSACIÓN")
    print("="*70)
    
    # Run the agent and append the exchange to the thread
    prompt = "Tell me a short pirate joke."
    print(f"\n👤 Usuario: {prompt}")
    
    response = await agent.run(prompt, thread=thread)
    print(f"\n🤖 Assistant: {response.text}")
    
    return response.text

print("✅ Función first_conversation() definida")

In [ ]:
# Ejecutar primera conversación
first_response = await first_conversation()

## 6. Serializar el Thread

### Serialización:
Convertir el estado del thread a un formato que se puede guardar.

### Método `.serialize()`:
```python
serialized_thread = await thread.serialize()
```

Retorna un diccionario con:
- Historial de mensajes
- Metadata del thread
- Estado de context providers (si los hay)

### Guardado:
Convertimos a JSON y guardamos en archivo.
En producción, guardarías en:
- Base de datos (CosmosDB, PostgreSQL)
- Blob storage (Azure Blob, S3)
- Cache (Redis)

In [ ]:
async def serialize_and_save():
    """Serializa el thread y lo guarda en archivo"""
    
    print("\n" + "="*70)
    print("SERIALIZACIÓN DEL THREAD")
    print("="*70)
    
    # Serialize the thread state
    print("\n🔄 Serializando thread...")
    serialized_thread = await thread.serialize()
    serialized_json = json.dumps(serialized_thread)
    
    # Example: save to a local file (replace with DB or blob storage in production)
    temp_dir = tempfile.gettempdir()
    file_path = os.path.join(temp_dir, "agent_thread.json")
    
    print(f"\n💾 Guardando thread en: {file_path}")
    with open(file_path, "w") as f:
        f.write(serialized_json)
    
    print("\n✅ Thread serializado y guardado")
    print(f"   Tamaño: {len(serialized_json)} caracteres")
    print(f"\n📊 Contenido del thread:")
    print(f"   {len(json.loads(serialized_json).get('messages', []))} mensajes guardados")
    
    return file_path

print("✅ Función serialize_and_save() definida")

In [ ]:
# Serializar y guardar
saved_path = await serialize_and_save()

## 7. Deserializar el Thread

### Deserialización:
Recrear el estado del thread desde el JSON guardado.

### Pasos:
1. Leer el JSON desde archivo
2. Parsear a diccionario
3. Usar `agent.deserialize_thread(data)` para recrear

### Importante:
- El thread deserializado debe usarse con el **mismo agente** (o uno equivalente)
- Contiene todo el historial de mensajes
- Permite continuar conversación exactamente donde quedó

In [ ]:
async def load_and_deserialize(file_path):
    """Carga y deserializa el thread desde archivo"""
    
    print("\n" + "="*70)
    print("DESERIALIZACIÓN DEL THREAD")
    print("="*70)
    
    # Read persisted JSON
    print(f"\n📂 Cargando thread desde: {file_path}")
    with open(file_path, "r") as f:
        loaded_json = f.read()

    reloaded_data = json.loads(loaded_json)
    
    print(f"\n🔄 Deserializando thread...")
    # Deserialize the thread into an AgentThread tied to the same agent type
    resumed_thread = await agent.deserialize_thread(reloaded_data)
    
    print("\n✅ Thread deserializado exitosamente")
    print(f"   {len(reloaded_data.get('messages', []))} mensajes recuperados")
    
    return resumed_thread

print("✅ Función load_and_deserialize() definida")

In [ ]:
# Deserializar el thread
resumed_thread = await load_and_deserialize(saved_path)

## 8. Continuar la Conversación

### Segunda Conversación:
Ahora continuamos con el thread deserializado.

### Mensaje 2:
"Now tell that joke in the voice of a pirate."

### Contexto preservado:
- El agente **recuerda** el chiste anterior
- Puede hacer referencia a la conversación previa
- El contexto se mantiene intacto

In [ ]:
async def continue_conversation(resumed_thread):
    """Continúa la conversación con el thread deserializado"""
    
    print("\n" + "="*70)
    print("CONTINUANDO LA CONVERSACIÓN")
    print("="*70)
    print("\n💡 Nota: El agente recuerda el chiste anterior gracias al thread deserializado\n")
    
    # Continue the conversation with resumed thread
    prompt = "Now tell that joke in the voice of a pirate."
    print(f"👤 Usuario: {prompt}")
    
    response = await agent.run(prompt, thread=resumed_thread)
    print(f"\n🤖 Assistant: {response.text}")
    
    print("\n✅ Conversación continuada exitosamente")
    print("   El agente usó el contexto del chiste anterior!")

print("✅ Función continue_conversation() definida")

In [ ]:
# Continuar conversación
await continue_conversation(resumed_thread)

## 9. Flujo Completo - Todo Junto

Aquí está el flujo completo ejecutado en una sola celda:

In [ ]:
async def complete_flow():
    """Ejecuta el flujo completo de persistencia"""
    
    print("\n" + "="*70)
    print("FLUJO COMPLETO DE PERSISTENCIA")
    print("="*70)
    
    # 1. Primera conversación
    await first_conversation()
    
    # 2. Serializar y guardar
    file_path = await serialize_and_save()
    
    # 3. Deserializar
    resumed = await load_and_deserialize(file_path)
    
    # 4. Continuar conversación
    await continue_conversation(resumed)
    
    print("\n" + "="*70)
    print("FLUJO COMPLETO FINALIZADO")
    print("="*70)

# Ejecutar flujo completo
# await complete_flow()

## Conclusión

Este ejemplo demuestra:

### 1. **Thread Serialization**
- `await thread.serialize()`: Exporta estado a diccionario
- JSON serialization para guardado
- Incluye mensajes y metadata

### 2. **Thread Deserialization**
- `await agent.deserialize_thread(data)`: Recrea thread
- Preserva contexto completo
- Permite continuar conversaciones

### 3. **Conversational Context**
- Thread mantiene historial de mensajes
- Referencias a conversaciones previas
- Contexto acumulado a través del tiempo

### 4. **Storage Flexibility**
- Ejemplo usa archivos temporales
- En producción: DB, blob storage, cache
- JSON portátil y fácil de migrar

### Aplicaciones Prácticas:

✅ **Long-running conversations**: Retomar después de días

✅ **Multi-channel**: Continuar conversación en diferentes dispositivos

✅ **Disaster recovery**: Restaurar estado después de crashes

✅ **Analytics**: Analizar patrones de conversación

✅ **A/B testing**: Comparar diferentes versiones de agentes

✅ **Training**: Usar conversaciones reales para fine-tuning

### Arquitectura de Persistencia:

#### Ejemplo con Cosmos DB:
```python
# Guardar
serialized = await thread.serialize()
await cosmos_client.upsert_item({
    'id': user_id,
    'thread_data': serialized,
    'timestamp': datetime.now()
})

# Cargar
doc = await cosmos_client.read_item(user_id)
thread = await agent.deserialize_thread(doc['thread_data'])
```

#### Ejemplo con Redis:
```python
# Guardar (con TTL de 24 horas)
serialized_json = json.dumps(await thread.serialize())
await redis.setex(f"thread:{user_id}", 86400, serialized_json)

# Cargar
data = json.loads(await redis.get(f"thread:{user_id}"))
thread = await agent.deserialize_thread(data)
```

### Consideraciones:

⚠️ **Tamaño**: Threads largos ocupan espacio

⚠️ **Privacy**: Datos sensibles en threads

⚠️ **Versioning**: Compatibilidad entre versiones de agente

⚠️ **TTL**: Definir tiempo de vida de threads

✅ **Compresión**: Comprimir JSON para storage eficiente

### Best Practices:

🔹 **Guardar después de cada interacción** (o batch)

🔹 **Incluir metadata**: user_id, timestamp, version

🔹 **Implementar cleanup**: Eliminar threads viejos

🔹 **Encriptar datos sensibles** en storage

🔹 **Backups regulares** de threads importantes

🔹 **Monitoring**: Track tamaño y frecuencia de guardado